<a href="https://colab.research.google.com/github/nandhiniswaminathan/face-recognition/blob/master/VGG_Resnet_FER.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install keras_vggface


In [ ]:
import numpy as np
import pandas as pd
import sys,os
from tensorflow.python.lib.io import file_io
from skimage.transform import resize
from keras import backend as K
from keras.utils import to_categorical
from keras_vggface.vggface import VGGFace
from keras.models import Model
from keras.layers import Flatten, Dense 
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam
from keras.callbacks import TensorBoard, LearningRateScheduler, ReduceLROnPlateau, EarlyStopping, Callback

Using TensorFlow backend.


In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
import sys,os
folder = "/content/drive/My Drive"
sys.path.insert(0, folder)
os.chdir(folder)

# Size of the images
img_height, img_width = 197, 197

# Parameters
num_classes         = 7     # ['Anger', 'Disgust', 'Fear', 'Happiness', 'Sadness', 'Surprise', 'Neutral']
epochs_top_layers   = 5
epochs_all_layers   = 50
batch_size          = 128

In [ ]:
filepath = "/content/drive/My Drive/Colab Notebooks/"
sys.path.insert(0, filepath)
os.chdir(filepath)


train_dir='/content/drive/My Drive/challenges-in-representation-learning-facial-expression-recognition-challenge (1).zip (Unzipped Files)/train.csv'
test_dir='test.csv'

In [ ]:
wdata = pd.read_csv(train_dir)
data=wdata.iloc[1:10000,]
ddt=wdata.iloc[10001:13000,]
gd=pd.read_csv(test_dir)

In [ ]:

test=wdata.iloc[13001:13050,]
#for better accuracy train on more data 

In [ ]:
base_model = VGGFace(
    model       = 'resnet50',
    include_top = False,
    weights     = 'vggface',
    input_shape = (img_height, img_width, 3))


x = base_model.output


x = Flatten()(x)

x = Dense(1024, activation = 'relu')(x)
predictions = Dense(num_classes, activation = 'softmax')(x)

# The model to be trained
model = Model(inputs = base_model.input, outputs = predictions)

94699520/94694792 [==============================] - 2s 0us/step


In [ ]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 197, 197, 3)  0                                            
__________________________________________________________________________________________________
conv1/7x7_s2 (Conv2D)           (None, 99, 99, 64)   9408        input_1[0][0]                    
__________________________________________________________________________________________________
conv1/7x7_s2/bn (BatchNormaliza (None, 99, 99, 64)   256         conv1/7x7_s2[0][0]               
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 99, 99, 64)   0           conv1/7x7_s2/bn[0][0]            
____________________________________________________________________________________________

In [ ]:
def preprocess_input(x):
    x -= 128.8006 # np.mean(train_dataset)
    return x

# Function to read the data from the csv file, increase the size of the images and return the images and their labels
def get_data(data):
    pixels = data['pixels'].tolist()
    images = np.empty((len(data), img_height, img_width, 3))
    i = 0

    for pixel_sequence in pixels:
        single_image = [float(pixel) for pixel in pixel_sequence.split(' ')]  # Extraction of each single
        single_image = np.asarray(single_image).reshape(48, 48) # Dimension: 48x48
        single_image = resize(single_image, (img_height, img_width), order = 3, mode = 'constant') # Dimension: 139x139x3 (Bicubic)
        ret = np.empty((img_height, img_width, 3))  
        ret[:, :, 0] = single_image
        ret[:, :, 1] = single_image
        ret[:, :, 2] = single_image
        images[i, :, :, :] = ret
        i += 1
    
    images = preprocess_input(images)
    labels = to_categorical(data['emotion'])

    return images, labels    

# Data preparation
train_data_x, train_data_y  = get_data(data)
val_data  = get_data(ddt)



In [ ]:
train_datagen = ImageDataGenerator(
    rotation_range  = 10,
    shear_range     = 10, # 10 degrees
    zoom_range      = 0.1,
    fill_mode       = 'reflect',
    horizontal_flip = True)

# Takes numpy data & label arrays, and generates batches of augmented/normalized data. Yields batcfillhes indefinitely, in an infinite loop
    # x:            Data. Should have rank 4. In case of grayscale data, the channels axis should have value 1, and in case of RGB data, 
    #               it should have value 3
    # y:            Labels
    # batch_size:   Int (default: 32)
train_generator = train_datagen.flow(
    train_data_x,
    train_data_y,
    batch_size  = batch_size)

In [ ]:
for layer in base_model.layers:
    layer.trainable = False

In [ ]:
model.compile(
    optimizer   = Adam(lr = 1e-3, beta_1 = 0.9, beta_2 = 0.999, epsilon = 1e-08, decay = 0.0), 
    loss        = 'categorical_crossentropy', 
    metrics     = ['accuracy'])

In [ ]:
tensorboard_top_layers = TensorBoard(
	log_dir         = folder + '/logs_top_layers',
	histogram_freq  = 0,
	write_graph     = True,
	write_grads     = False,
	write_images    = True)

In [ ]:
model.fit_generator(
    generator           = train_generator,
    steps_per_epoch     = len(train_data_x) // batch_size,  # samples_per_epoch / batch_size
    epochs              = 30,                            
    validation_data     = val_data) 



Epoch 1/30
78/78 [==============================] - 242s 3s/step - loss: 1.9621 - accuracy: 0.4212 - val_loss: 1.5588 - val_accuracy: 0.4542
Epoch 2/30
78/78 [==============================] - 252s 3s/step - loss: 1.1371 - accuracy: 0.5723 - val_loss: 1.3844 - val_accuracy: 0.5195
Epoch 3/30
 4/78 [>.............................] - ETA: 2:58 - loss: 1.0662 - accuracy: 0.5957

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
images, labels = get_data(test)
scores = model.evaluate(np.array(images), np.array(labels), batch_size=batch_size)
print("Loss: " + str(scores[0]))
print("Accuracy: " + str(scores[1]))

49/49 [==============================] - 1s 21ms/step
Loss: 2.214491844177246
Accuracy: 0.5306122303009033
